# Testing Rename Tool to get to change output from Paligo
This notebook is used to create and test the rename tool to support our naming convention.
Start: 2022-07-26
End:


- [x] search through the download folder with file name that starts with 4~5 digit of number and end with .zip
- [x] unzip the folder and read in the pdf 

- [x] get the file name from metadata or read in the header of the 1st page
- [x] get the MRA ref number from the front page
- [x] arrange the file name
- [x] save to new PDF



In [11]:
import os
from datetime import datetime
import PyPDF2
from zipfile import ZipFile
from PyPDF2 import PdfReader 
import shutil

In [ ]:
if os.getcwd() != '/Users/yi-chunwang/OneDrive - Perspectum Ltd/Work_Repo/TechWriting':

    os.chdir('/Users/yi-chunwang/OneDrive - Perspectum Ltd/Work_Repo/TechWriting')

today = datetime.today().strftime('%Y-%m-%d')
print('Last update on:', today)

## Build a function to choose where to save the output file

In [ ]:
PATH = input('Choose a folder/directory to save the output')
Download_PATH = input('Choose choose your download folder')

## Build RenameTool for English documents

In [ ]:
os.chdir('/Users/yi-chunwang/Downloads') # change the current directory to Downloads folder
downloads = os.listdir()

for download in downloads:
    if download[0].isdigit() and download.endswith('.zip'):
        zipfile = os.path.abspath(download) # get the absolute path of the zip file
        #print(zipfile)

        with ZipFile(zipfile) as zipObj:
            list_of_file_names = zipObj.namelist()
            #print(list_of_file_names)
            for fileName in list_of_file_names:
                if fileName.endswith('.pdf'):
                    zipObj.extract(fileName, '/Users/yi-chunwang/Downloads/')
                    # print(list_of_file_names[-1]) # 34495-ALT-801-106_Patient_Acquisition_Manual-pdf/out/ALT-801-106_Patient_Acquisition_Manual-en.pdf
                    # print(list_of_file_names)
                    target_name = '/Users/yi-chunwang/Downloads/'+ list_of_file_names[-1]
                    #print(target_name)

                    reader = PdfReader(target_name) # read the pdf
                    meta = reader.metadata # read the meta data
                    file_name = meta.title # get the proper file name from the title in meta data
                    page = reader.pages[0] # get the fron page
                    front_page_text = page.extract_text() # get the front page text and store as a variable
                    reference_start = front_page_text.find('MRA') # find the MRA reference number
                    MRAref = front_page_text[reference_start:reference_start+7] # store the MRA reference number as a variable
                    new_name =  file_name +' - '+ MRAref # compose new file name
                    os.rename(target_name, '/Users/yi-chunwang/Downloads/'+new_name+'.pdf')# rename the file with the new file name

                    # remove the temporary directory
                    shutil.rmtree(list_of_file_names[0])

## Build RenameTool for translated manuals

In [10]:
os.chdir('/Users/yi-chunwang/Downloads') # change the current directory to Downloads folder
downloads = os.listdir()

# build a language code and full language name mapping
lang_dict={
'-af': ' (Afrikaans)'
, '-ar': ' (Albanian)'
, '-be': ' (Baelarusian)'
, '-bg': ' (Bugarian)'
, '-bs': ' (Bosnian)'
, '-ca': ' (Catalan)'
, '-cs': ' (Czech)'
, '-da': ' (Danish)'
, '-de': ' (German)'
, '-el': ' (Greek)'
, '-en': ' (English)'
, '-eo': ' (Esperanto)'
, '-es': ' (Spanish)'
, '-et': ' (Estonian)'
, '-eu': ' (Basque)'
, '-fa': ' (Persian)'
, '-fi': ' (Finnish)'
, '-fr': ' (French)'
, '-he': ' (Hebrew)'
, '-hi': ' (Hindi)'
, '-hr': ' (Croatian)'
, '-hu': ' (Hangarain)'
, '-is': ' (Icelandic)'
, '-it': ' (Italian)'
, '-ja': ' (Japanese)'
, '-ji': ' (Yiddish)'
, '-ko': ' (Korean)'
, '-lt': ' (Lithuanian)'
, '-lv': ' (Latvian)'
, '-mg': ' (Malagasy)'
, '-mk': ' (Macedonian)'
, '-nb': ' (Norwegian Bokmål)'
, '-nl': ' (Dutch)'
, '-nn': ' (Norwegian Nynorsk)'
, '-no': ' (Norwegian)'
, '-pl': ' (Polish)'
, '-pt': ' (Portuguese)'
, '-ro': ' (Romanian)'
, '-ru': ' (Russian)'
, '-sk': ' (Slovak)'
, '-sl': ' (Slovenian)'
, '-sq': ' (Albanian)'
, '-sr': ' (Serbian)'
, '-sv': ' (Swedish)'
, '-th': ' (Thai)'
, '-tr': ' (Turkish)'
, '-uk': ' (Ukranian)'
, '-vi': ' (Vietnamese)'
, '-za': ' (Zhuang)'
, '-zh': ' (Chinese)'
, '-zh-chs': ' (Chinese-simplified)'
, '-zh-cht': ' (Chinese-traditional)'
}


for download in downloads:
    if download[0].isdigit() and download.endswith('.zip'):
        zipfile = os.path.abspath(download) # get the absolute path of the zip file
        with ZipFile(zipfile) as zipObj:
          list_of_file_names = zipObj.namelist() # get a list of file names of entire folder tree (from top zip folder to inner most files)
          for fileName in list_of_file_names:
                if fileName.endswith('.pdf'):
                  zipObj.extract(fileName, '/Users/yi-chunwang/Downloads/')
                  target_name = '/Users/yi-chunwang/Downloads/'+ list_of_file_names[-1]
                  # print(target_name)
                  index_start = list_of_file_names[-1].find('out/') # get the starting index of the last item in the list
                  file_name_strip = list_of_file_names[-1][index_start+4:].replace('_', ' ').replace('.pdf', '') # extract out the file name itseld and replace underscore with space
          
        # Replace language code with full language 
        for key in lang_dict.keys():
          file_name_strip = file_name_strip.replace(key, lang_dict[key]) # replace the language code with full labguage name
          
        # Add vendor name into file name
        split_name = file_name_strip.split('for')
        reader = PdfReader(target_name)
        title = reader.metadata.title 
        if title.find('Siemens') != -1:
          insert_vendor = split_name[0]+'for Siemens'+ split_name[1]
        elif title.find('GE') != -1:
          insert_vendor = split_name[0]+'for GE'+ split_name[1]
        else:
          insert_vendor = split_name[0]+'for Philips'+ split_name[1]
        #print(insert_vendor)

        # Add MRA ref
        page = reader.pages[0] # get the front page
        front_page_text = page.extract_text() # get the front page text and store as a variable
        reference_start = front_page_text.find('MRA') # find the MRA reference number
        MRAref = front_page_text[reference_start:reference_start+7] # store the MRA reference number as a variable
        new_name =  insert_vendor +' - '+ MRAref # compose new file name
        os.rename(target_name, '/Users/yi-chunwang/Downloads/'+new_name+'.pdf') # rename the file with the new file name

        # remove the temporary directory
        shutil.rmtree(list_of_file_names[0])

In [ ]:
# No need tp unpack because the ZipFile function had supported the unzip process, and it also provides us with the downstream file name
 
shutil.unpack_archive(zipfile, '/Users/yi-chunwang/Downloads/', 'zip') #use function to unzip file
os.chdir()
shutil.unpack_archive(zipfile, '/Users/yi-chunwang/Downloads/', 'zip') #use function to unzip file
os.mv
print('unzip done')

In [ ]:

# there is no need to walk now because the target_name had been find out by the list of filename from the ZipFile function

for root, dirs, files in os.walk('/Users/yi-chunwang/Downloads/out/34495-ALT-801-106_Patient_Acquisition_Manual-pdf', topdown=False):
    for name in files:
        if name.endswith('.pdf'):
            target_name = os.path.join(root, name)
